In [10]:

import pandas as pd
import numpy as np
pd.set_option('max_columns', 50, 'max_rows', 50)

In [11]:
data = pd.read_csv("datasets/store.csv")
data.head()

,RowID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductID,Category,Sub-Category,ProductName,Sales,Quantity,Discount,Profit,Cost_Price
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,65.49000
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,121.99000
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,3.65500
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,95.75775
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,5.59200


In [12]:
data.columns

Index(['RowID', 'OrderID', 'OrderDate', 'ShipDate', 'ShipMode', 'CustomerID',
       'CustomerName', 'Segment', 'Country', 'City', 'State', 'PostalCode',
       'Region', 'ProductID', 'Category', 'Sub-Category', 'ProductName',
       'Sales', 'Quantity', 'Discount', 'Profit', 'Cost_Price'],
      dtype='object')

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 22 columns):
RowID           9994 non-null int64
OrderID         9994 non-null object
OrderDate       9994 non-null object
ShipDate        9994 non-null object
ShipMode        9994 non-null object
CustomerID      9994 non-null object
CustomerName    9994 non-null object
Segment         9994 non-null object
Country         9994 non-null object
City            9994 non-null object
State           9994 non-null object
PostalCode      9994 non-null int64
Region          9994 non-null object
ProductID       9994 non-null object
Category        9994 non-null object
Sub-Category    9994 non-null object
ProductName     9994 non-null object
Sales           9994 non-null float64
Quantity        9994 non-null int64
Discount        9994 non-null float64
Profit          9994 non-null float64
Cost_Price      9994 non-null float64
dtypes: float64(4), int64(3), object(15)
memory usage: 1.7+ MB


In [14]:
print(data["Country"].unique())

['United States']


In [15]:
print(data["State"].unique())

['Kentucky' 'California' 'Florida' 'North Carolina' 'Washington' 'Texas'
 'Wisconsin' 'Utah' 'Nebraska' 'Pennsylvania' 'Illinois' 'Minnesota'
 'Michigan' 'Delaware' 'Indiana' 'New York' 'Arizona' 'Virginia'
 'Tennessee' 'Alabama' 'South Carolina' 'Oregon' 'Colorado' 'Iowa' 'Ohio'
 'Missouri' 'Oklahoma' 'New Mexico' 'Louisiana' 'Connecticut' 'New Jersey'
 'Massachusetts' 'Georgia' 'Nevada' 'Rhode Island' 'Mississippi'
 'Arkansas' 'Montana' 'New Hampshire' 'Maryland' 'District of Columbia'
 'Kansas' 'Vermont' 'Maine' 'South Dakota' 'Idaho' 'North Dakota'
 'Wyoming' 'West Virginia']


In [16]:
#We have so many state, let us see how sales perform in each state
state_sales = data.groupby("State")['Sales'].sum().reset_index()
state_sales.head()

,State,Sales
0,Alabama,19510.6400
1,Arizona,35282.0010
2,Arkansas,11678.1300
3,California,457687.6315
4,Colorado,32108.1180


In [17]:
# let us find the state with lowest and highest sale)
print(state_sales[state_sales.Sales == state_sales.Sales.max()])
print(state_sales[state_sales.Sales == state_sales.Sales.min()])


        State        Sales
3  California  457687.6315
           State   Sales
32  North Dakota  919.91


In [18]:
"""
So we have the state with lowest sale is North Dakota and highest sale is California
We're going to work with these 2 states for now and implement all our kpi metrics feature and
discover why one state is performing low compared to another, note you can implement these 
with all the states.
"""

north_dakota= data.query("State=='North Dakota'").reset_index(drop=True)
california = data.query("State=='California'").reset_index(drop=True)
# alabama_data.head().append(wyoming_data.tail())


In [19]:
#let us get to action, write functions for all our kpis

In [20]:
def process_date(df):
    #to avoid KeyError, reset the index
    df = df.reset_index()
    #convert date strings to datetime
    df['OrderDate'] = pd.to_datetime(df['OrderDate'])
    df = df.set_index('OrderDate').sort_index()
    return df


In [21]:
def monthly_store_revenue(df):
    df = process_date(df)
    monthly_revenue = df.resample('M')['Sales'].sum().reset_index()
    #calculate monthly growth (MoM)
    monthly_revenue = monthly_revenue.assign(MoM=lambda x:x['Sales'].pct_change())
    return monthly_revenue

def yearly_active_customers(df):
    df = process_date(df)
    yearly_customers = df.resample('Y')['CustomerID'].nunique().reset_index()
    return yearly_customers

def monthly_active_customers(df):
    df = process_date(df)
    monthly_customers = df.resample('M')['CustomerID'].nunique()
    return monthly_customers


In [22]:
#how often people buy from the store
def monthly_order_count(df):
    df = process_date(df)
    monthly_order = df.resample('M')['Quantity'].sum().reset_index()
    return monthly_order

In [23]:
def store_conversion_rate(df):
    # Total number of visitors over number of conversions
    visitors = len(df["CustomerID"])
    conversions= (df['Sales'].sum()/len(df["CustomerID"].unique())) * 100
    conversion_rate = (visitors /conversions)* 100
    return conversion_rate
    
    
    
def churn_rate(df):
        #calcualate customer churn rate for 1 year
        df = process_date(df)
        clients_begin = df. first(pd.offsets.MonthBegin(3, normalize=True))
        cb = len(clients_begin['CustomerID'].unique())

        clients_end = df.first(pd.offsets.MonthEnd(3))
        ce = len(clients_end['CustomerID'].unique())

        churn_rate = (cb -ce) /cb * 100
        return churn_rate
    
def monthly_average_order_value(df):
        """Average order value – also know as AOV – is an ecommerce metric 
        that refers to the average amount of money spent by customers per order.
        Total Revenue ÷ Total No. of Orders = Average Order Value"""
        
        avg_order = process_date(df)
        avg_order['average_order'] = avg_order['Sales']/avg_order['Quantity']
        avg_order = avg_order.resample('M')['average_order'].sum().reset_index()
        return avg_order
    
def purchase_frequency(df):
        # PF = Total number of orders/ Total number of unique customers.
        repeat_customers = len(df['CustomerID'].unique())
        customer_purchase_frequency = df['Quantity'].sum()/repeat_customers
        return customer_purchase_frequency
    
    
def gross_profit_margin(df):
        # Gross profit margin = gross profit (revenue – cost of goods sold) / revenue
        gross_profit = df['Sales'].sum() - df['Cost_Price'].sum()
        margin_profit = (gross_profit /df['Sales'].sum()) * 100
        return margin_profit
    

def time_between_purchase(df):
    purchase_freq = purchase_frequency(df)
    time_btw_purchase = purchase_freq/(365)
    return time_btw_purchase

def repeat_purchase_rate(df):
        # RPR = Purchase from repeat customers/Total purchases
        repeat_customers = len(df['CustomerID'].unique())
        customer_repeat_purchase =  repeat_customers /df['Sales'].sum()
        return customer_repeat_purchase * 100
    

def customer_lifetime_value(data):
        """ Calculate the customer lifetime value for 6 months
        
        customer_lifetime = (average_transactions * average_order_value 
                             *average_gross_margin*average_lifespan)
                             /no_of_customers
        """

        avg_trans = avg_transactions(data)
        avg_order_value = avg_order(data)
        avg_gross  = avg_gross_margin(data)
        avg_lifespan = average_life_span(data)
        no_of_customers = number_of_clients(data)

        customer_lifetime = (avg_trans * avg_order_value *avg_gross*avg_lifespan)\
                            / no_of_customers 
        return customer_lifetime


def average_life_span(df):
        #calculate customer average lifespan
        
        df = churn_rate(df)
        average_lifespan = 1/df
        return average_lifespan
        
def avg_gross_margin(df):
        #calculate average gross margin profit for the store
        df = process_date(df)
        average_order = df.resample('M')['Sales','Cost_Price'].sum()
        average_order['gross_margin'] = (average_order['Sales'] - average_order['Cost_Price'])\
                                                                 /average_order['Sales'] *100
        average_gross_margin = average_order['gross_margin'].sum()/len(average_order['gross_margin'])
        return average_gross_margin             
        
def avg_order(df):
        #calculate an singular average order for the store
        average_order_value = df['Sales'].sum()/df['Quantity'].sum()
        return average_order_value        
        
def number_of_clients(df):
        #calculate the total number of customers that visit the store
        num_of_clients = len(df['CustomerID'].unique())
        return num_of_clients
def avg_transactions(df):
        #calculate store average transaction every 1 year
        average_transactions= df['Quantity'].sum()/12
        return average_transactions
    

In [24]:
# 1. AVERAGE ORDER VALUE (AOV)
California = monthly_average_order_value(california)
North_dakota = monthly_average_order_value(north_dakota)
print(California.head())
print(North_dakota.head())

   OrderDate  average_order
0 2014-01-31        728.966
1 2014-02-28        125.030
2 2014-03-31       1808.174
3 2014-04-30       2063.375
4 2014-05-31       1591.427
   OrderDate  average_order
0 2017-05-31         231.27
1 2017-06-30           0.00
2 2017-07-31           0.00
3 2017-08-31           0.00
4 2017-09-30           0.00


In [36]:
# 2.CUSTOMER LIFE TIME VALUE FOR 6 MONTHS
California = (process_date(california)).loc[:'2014-06-30']
print(customer_lifetime_value(California))
print("\n")
# north_dakota = (process_date(north_dakota)).loc[:'2014-06-30']
# print(customer_lifetime_value(north_dakota))
store_data = (process_date(data)).loc[:'2014-06-30']
print(customer_lifetime_value(store_data))

840.3260491239907


5060.387471189002


In [26]:
#3.CHURN RATE FOR 6 MONTHS
California = (process_date(california)).loc[:'2014-06-30']
North_dakota = (process_date(north_dakota)).loc[:'2014-06-30']
print(churn_rate(California))
# print(churn_rate(North_dakota))


4.0


In [27]:
#4. CUSTOMER REPEAT PURCHASE RATE
California = repeat_purchase_rate(california)
North_dakota = repeat_purchase_rate(north_dakota)
Store_data = repeat_purchase_rate(data)

print(California)
print(North_dakota)
print(Store_data)

0.12606851491900103
0.2174125729690948
0.03452027263721463


In [29]:
#5.CUSTOMER PURCHASE FREQUENCY

print(purchase_frequency(california))
print(purchase_frequency(north_dakota))

13.287694974003466
15.0


In [30]:
#6. STORE MONTHLY REVENUE

California = monthly_store_revenue(california)
North_dakota = monthly_store_revenue(north_dakota)
print(California.head())
print(North_dakota.head())

   OrderDate     Sales        MoM
0 2014-01-31  2455.185        NaN
1 2014-02-28   308.702  -0.874265
2 2014-03-31  7239.096  22.450110
3 2014-04-30  8165.181   0.127928
4 2014-05-31  5960.020  -0.270069
   OrderDate   Sales  MoM
0 2017-05-31  891.53  NaN
1 2017-06-30    0.00 -1.0
2 2017-07-31    0.00  NaN
3 2017-08-31    0.00  NaN
4 2017-09-30    0.00  NaN


In [31]:
# 7.MONTHLY ORDER COUNT
California= monthly_order_count(california)
North_dakota = monthly_order_count(north_dakota)
monthly_order_count(data)
print(California.head())
print(North_dakota.head())

   OrderDate  Quantity
0 2014-01-31        30
1 2014-02-28        21
2 2014-03-31        78
3 2014-04-30       145
4 2014-05-31        99
   OrderDate  Quantity
0 2017-05-31        24
1 2017-06-30         0
2 2017-07-31         0
3 2017-08-31         0
4 2017-09-30         0


In [32]:
#8.CONVERSION RATE
print(store_conversion_rate(california))
print(store_conversion_rate(north_dakota))

2.522630983529211
0.015218880107836636


In [33]:
#9.CALCULATE ACTIVE CUSTOMERS
print(yearly_active_customers(california).head())
print("\n")
print(yearly_active_customers(north_dakota).head())

   OrderDate  CustomerID
0 2014-12-31         181
1 2015-12-31         185
2 2016-12-31         235
3 2017-12-31         291


   OrderDate  CustomerID
0 2017-12-31           2


In [34]:
#10.GROSS PROFIT MARGIN
print(avg_gross_margin(california))
print(avg_gross_margin(north_dakota))

86.35165596411832
21.691772686156785
